***
# 数据抓取：

> # 使用Python编写网络爬虫
***

王成军

wangchengjun@nju.edu.cn

计算传播网 http://computational-communication.com

modified by liding@ruc.edu.cn to python3  2019-12-10

# 需要解决的问题 

- 页面解析
- 获取Javascript隐藏源数据
- 自动翻页
- 自动登录
- 连接API接口


In [7]:
import requests
from bs4 import BeautifulSoup

- 一般的数据抓取，使用urllib2和beautifulsoup配合就可以了。在python3 中可以使用requests
- 尤其是对于翻页时url出现规则变化的网页，只需要处理规则化的url就可以了。
- 以简单的例子是抓取天涯论坛上关于某一个关键词的帖子。
    - 在天涯论坛，关于雾霾的帖子的第一页是：
http://bbs.tianya.cn/list.jsp?item=free&nextid=0&order=8&k=雾霾
    - 第二页是：
http://bbs.tianya.cn/list.jsp?item=free&nextid=1&order=8&k=雾霾


***
# 数据抓取：
> # 抓取天涯回帖网络
***

王成军

wangchengjun@nju.edu.cn

计算传播网 http://computational-communication.com

In [1]:
from IPython.display import display_html, HTML
HTML('<iframe src=http://bbs.tianya.cn/list.jsp?item=free&nextid=%d&order=8&k=PX width=1000 height=500></iframe>')
# the webpage we would like to crawl

/Users/liding/anaconda/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [10]:
page_num = 0
url = "http://bbs.tianya.cn/list.jsp?item=free&nextid=%d&order=8&k=PX"% page_num
content = requests.get(url) #获取网页的html文本
soup = BeautifulSoup(content.text, "lxml") 
articles = soup.find_all('tr')

In [12]:
print (articles[0])

<tr>
<th scope="col"> 标题</th>
<th scope="col">作者</th>
<th scope="col">点击</th>
<th scope="col">回复</th>
<th scope="col">发表时间</th>
</tr>


In [13]:
print (articles[1])

<tr class="bg">
<td class="td-title facediamond">
<a class="diamond" href="http://bei.tianya.cn/wallet/zuan.do" target="_blank" title="钻石榜"></a>
<a class="" href="http://bbs.tianya.cn/post-free-6101505-1.shtml" target="_blank">违反消防法第十三条，难道反而成了有理的吗？法制何在？</a>
</td>
<td><a class="author" href="http://www.tianya.cn/134144947" target="_blank">风轻云淡求公正</a></td>
<td>20107</td>
<td>514</td>
<td title="null">12-10 09:16</td>
</tr>


In [14]:
len(articles[1:])

2

![](./img/inspect.png)

http://bbs.tianya.cn/list.jsp?item=free&nextid=0&order=8&k=PX
    
# 通过分析帖子列表的源代码，使用inspect方法，会发现所有要解析的内容都在‘td’这个层级下
    

In [31]:
for t in articles[1].find_all('td'): print (t)

<td class="td-title facediamond">
<a class="diamond" href="http://bei.tianya.cn/wallet/zuan.do" target="_blank" title="钻石榜"></a>
<a class="" href="http://bbs.tianya.cn/post-free-5823246-1.shtml" target="_blank">4乘1.5=3.5乘1.2？ 快看山东济南市历下区法院是否判决不公？</a>
</td>
<td><a class="author" href="http://www.tianya.cn/134144947" target="_blank">风轻云淡求公正</a></td>
<td>100783</td>
<td>3350</td>
<td title="null">12-11 09:20</td>


In [17]:
td = articles[1].find_all('td')

In [18]:
print (td[0])

<td class="td-title facediamond">
<a class="diamond" href="http://bei.tianya.cn/wallet/zuan.do" target="_blank" title="钻石榜"></a>
<a class="" href="http://bbs.tianya.cn/post-free-6101505-1.shtml" target="_blank">违反消防法第十三条，难道反而成了有理的吗？法制何在？</a>
</td>


In [19]:
print (td[0])

<td class="td-title facediamond">
<a class="diamond" href="http://bei.tianya.cn/wallet/zuan.do" target="_blank" title="钻石榜"></a>
<a class="" href="http://bbs.tianya.cn/post-free-6101505-1.shtml" target="_blank">违反消防法第十三条，难道反而成了有理的吗？法制何在？</a>
</td>


In [20]:
print (td[0].text)



违反消防法第十三条，难道反而成了有理的吗？法制何在？



In [21]:
print (td[0].text.strip())

违反消防法第十三条，难道反而成了有理的吗？法制何在？


In [22]:
print (td[0].a['href'])

http://bei.tianya.cn/wallet/zuan.do


In [23]:
print (td[1])

<td><a class="author" href="http://www.tianya.cn/134144947" target="_blank">风轻云淡求公正</a></td>


In [24]:
print (td[2])

<td>20107</td>


In [17]:
print (td[3])

<td>1041</td>


In [25]:
print (td[4])

<td title="null">12-10 09:16</td>


In [28]:
records = []
for i in articles[1:]:
    td = i.find_all('td')
    title = td[0].text.strip()
    title_url = td[0].a['href']
    author = td[1].text
    author_url = td[1].a['href']
    views = td[2].text
    replies = td[3].text
    date = td[4]['title']
    record = title + '\t' + title_url+ '\t' + author + '\t'+ author_url + '\t' + views+ '\t'  + replies+ '\t'+ date
    records.append(record)

In [32]:
print (records[1])

4乘1.5=3.5乘1.2？ 快看山东济南市历下区法院是否判决不公？	http://bei.tianya.cn/wallet/zuan.do	风轻云淡求公正	http://www.tianya.cn/134144947	100783	3350	null


# 抓取天涯论坛PX帖子列表

回帖网络（Thread network）的结构
- 列表
- 主帖
- 回帖

In [51]:
def crawler(page_num, file_name):
    try:
        # open the browser
        url = "http://bbs.tianya.cn/list.jsp?item=free&nextid=%d&order=8&k=PX" % page_num
        content = requests.get(url).text #获取网页的html文本
        soup = BeautifulSoup(content, "lxml") 
        articles = soup.find_all('tr')
        # write down info
        for i in articles[1:]:
            td = i.find_all('td')
            title = td[0].text.strip()
            title_url = td[0].a['href']
            author = td[1].text
            author_url = td[1].a['href']
            views = td[2].text
            replies = td[3].text
            date = td[4]['title']
            record = title + '\t' + title_url+ '\t' + author + '\t'+ \
                        author_url + '\t' + views+ '\t'  + replies+ '\t'+ date
            with open(file_name,'a',encoding='utf-8') as p: # '''Note'''：Ａppend mode, run only once!
                        p.write(record+"\n") ##!!encode here to utf-8 to avoid encoding
                    
    except Exception as e:
        print (e)
        pass

In [53]:
# crawl all pages
for page_num in range(10):
    print (page_num) 
    crawler(page_num, '/Users/liding/E/Bdata/2019R/l14spider/python/tianya_bbs_threads_list2017.txt') 

0
1
2
3
4
5
6
7
8
9


In [54]:
import pandas as pd

df = pd.read_csv('/Users/liding/E/Bdata/2019R/l14spider/python/tianya_bbs_threads_list2017.txt',\
                 sep = "\t",  names = ['title', 'link', 'author', \
                 'author_page', 'click', 'reply', 'time'])
df[:2]

,title,link,author,author_page,click,reply,time
0,违反消防法第十三条，难道反而成了有理的吗？法制何在？,http://bei.tianya.cn/wallet/zuan.do,风轻云淡求公正,http://www.tianya.cn/134144947,20107,514,NaN
1,4乘1.5=3.5乘1.2？ 快看山东济南市历下区法院是否判决不公？,http://bei.tianya.cn/wallet/zuan.do,风轻云淡求公正,http://www.tianya.cn/134144947,100783,3350,NaN


In [55]:
len(df)

454

In [56]:
len(df.link)

454

# 抓取作者信息

In [65]:
df.author_page[:5]

0    http://www.tianya.cn/134144947
1    http://www.tianya.cn/134144947
2     http://www.tianya.cn/36959960
3     http://www.tianya.cn/16142449
4     http://www.tianya.cn/50499450
Name: author_page, dtype: object

http://www.tianya.cn/62237033
    
http://www.tianya.cn/67896263
    

In [58]:
# user_info

In [69]:
url = df.author_page[4]
content = requests.get(url).text #获取网页的html文本
soup = BeautifulSoup(content, "lxml") 
soup

<!DOCTYPE HTML>
<!--[if IE 6]><html class="ie6 lte9 lte8 lte7" lang="zh-cn"><![endif]--><!--[if IE 8]><html class="ie8 lte9 lte8" lang="zh-cn"><![endif]--><!--[if IE 9]><html class="ie9 lte9" lang="zh-cn"><![endif]--><!--[if IE 7]><html class="ie7 lte9 lte8 lte7" lang="zh-cn"><![endif]--><!--[if !(IE 6) | !(IE 7) | !(IE 8) | !(IE 9)  ]><!--><html lang="zh-cn"><!--<![endif]-->
<head>
<meta charset="utf-8"/>
<title>贾也_天涯社区</title>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="Thu, 01 Nov 2012 10:00:00 GMT" http-equiv="Expires"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<link href="http://static.tianyaui.com/global/ty/TY.css" rel="stylesheet" type="text/css"/>
<link href="http://static.tianyaui.com/global/my/web/static/css/main_8af4331.css" rel="stylesheet" type="text/css"/>
<link href="http://static.tianyaui.com/global/my/web/static/css/t_401edc9.css" rel="stylesheet" type="text/css"/>
<link href="http://static.tianyaui.com/favicon.ico" rel="shortcut icon" t

In [78]:
user_info = soup.find('div',  {'class', 'info-wrapper'})('li')
for i in user_info:
    print (i.get_text()[0:])

浙江杭州市
70后 
门中木人一枚。公众号：meiriguanjian 微信号：jiaye4547
标签
              文史自修研究生 驴友闪走族 经济自由主义者 文化保守主义 码字发条 设计 原创 读书


In [79]:
user_info = soup.find('div',  {'class', 'info-wrapper'})('li')
nid, freq_use, last_login_time, reg_time = \
[i.get_text()[0:] for i in user_info]
print (nid, freq_use, last_login_time, reg_time)

link_info = soup.find_all('div', {'class', 'link-box'})
followed_num, fans_num = [i.a.text for i in link_info]
print (followed_num, fans_num)

浙江杭州市 70后  门中木人一枚。公众号：meiriguanjian 微信号：jiaye4547 标签
              文史自修研究生 驴友闪走族 经济自由主义者 文化保守主义 码字发条 设计 原创 读书
154 28304


In [81]:
activity = soup.find_all('span', {'class', 'subtitle'})
post_num, reply_num = [j.text[2:] for i in activity[:1] for j in i('a')]
print (post_num, reply_num)

1 12


In [83]:
print (activity[0])

<span class="subtitle"><a href="http://www.tianya.cn/50499450/group?t=post" target="_blank">主帖1</a>　<a href="http://www.tianya.cn/50499450/group?t=reply" target="_blank">回帖12</a></span>


In [84]:
link_info = soup.find_all('div', {'class', 'link-box'})
followed_num, fans_num = [i.a.text for i in link_info]
print (followed_num, fans_num)

154 28304


In [85]:
link_info[0].a.text

'154'

In [86]:
# user_info = soup.find('div',  {'class', 'userinfo'})('p')
# user_infos = [i.get_text()[4:] for i in user_info]
            
def author_crawler(url, file_name):
    try:
        content = requests.get(url).text#获取网页的html文本
        soup = BeautifulSoup(content, "lxml")
        link_info = soup.find_all('div', {'class', 'link-box'})
        followed_num, fans_num = [i.a.text for i in link_info]
        try:
            activity = soup.find_all('span', {'class', 'subtitle'})
            post_num, reply_num = [j.text[2:] for i in activity[:1] for j in i('a')]
        except:
            post_num, reply_num = 1, 0
        record =  '\t'.join([url, followed_num, fans_num, post_num, reply_num])
        with open(file_name,'a',encoding='utf-8') as p: # '''Note'''：Ａppend mode, run only once!
                    p.write(record +"\n") ##!!encode here to utf-8 to avoid encoding

    except Exception as e:
        print (e, url)
        record =  '\t'.join([url, 'na', 'na', 'na', 'na'])
        with open(file_name,'a',encoding='utf-8') as p: # '''Note'''：Ａppend mode, run only once!
                    p.write(record +"\n") ##!!encode here to utf-8 to avoid encoding
        pass

In [87]:
import sys
def flushPrint(s):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % s)
    sys.stdout.flush()

In [88]:
import time, random
for k, url in enumerate(df.author_page):
    time.sleep(random.random())
    if k % 10==0:
        flushPrint(k)
    author_crawler(url, '/Users/liding/E/Bdata/2019R/l14spider/python/tianya_bbs_threads_author_info2017_1.txt') 

0not enough values to unpack (expected 2, got 0) http://www.tianya.cn/36959960
sequence item 3: expected str instance, int found http://www.tianya.cn/25210087
sequence item 3: expected str instance, int found http://www.tianya.cn/3151104
sequence item 3: expected str instance, int found http://www.tianya.cn/68696683
10not enough values to unpack (expected 2, got 0) http://www.tianya.cn/19265466
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/44156220
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/7260262
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/81120713
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/80671380
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/44156220
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/115073576
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/80982782
not enough values to unpack (expected 2, got 0) http://w

not enough values to unpack (expected 2, got 0) http://www.tianya.cn/79139119
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/83758764
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/81126787
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/87950543
120not enough values to unpack (expected 2, got 0) http://www.tianya.cn/85055580
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/74840170
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/80463015
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/85332091
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/84874073
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/84874073
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/82715311
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/78013536
not enough values to unpack (expected 2, got 0) http://www.ti

not enough values to unpack (expected 2, got 0) http://www.tianya.cn/80982782
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/67896263
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/80449763
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/83688158
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/85363102
230not enough values to unpack (expected 2, got 0) http://www.tianya.cn/74840170
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/54533776
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/80463015
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/n/hglfjnp
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/85332091
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/84874073
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/65954376
not enough values to unpack (expected 2, got 0) http://www.t

not enough values to unpack (expected 2, got 0) http://www.tianya.cn/84874073
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/84874073
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/85378823
340not enough values to unpack (expected 2, got 0) http://www.tianya.cn/83150488
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/84874073
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/55420492
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/39276289
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/80969329
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/n/MX147258
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/83666971
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/82679836
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/85356930
not enough values to unpack (expected 2, got 0) http://www.

not enough values to unpack (expected 2, got 0) http://www.tianya.cn/84708627
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/83175791
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/85359493
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/83134734
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/134144947
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/134144947
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/134144947
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/134144947
450not enough values to unpack (expected 2, got 0) http://www.tianya.cn/134144947
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/134144947


http://www.tianya.cn/50499450/follow  

还可抓取他们的关注列表和粉丝列表

***
***
# 数据抓取：
> # 使用Python抓取回帖
***
***

王成军

wangchengjun@nju.edu.cn

计算传播网 http://computational-communication.com

In [62]:
df.link[0]

'/post-free-2839539-1.shtml'

In [96]:
url = 'http://bbs.tianya.cn' + df.link[2]
print url

http://bbs.tianya.cn/post-free-2848995-1.shtml


In [64]:
from IPython.display import display_html, HTML
HTML('<iframe src=http://bbs.tianya.cn/post-free-2848797-1.shtml width=1000 height=500></iframe>')
# the webpage we would like to crawl

In [65]:
post = urllib2.urlopen(url).read() #获取网页的html文本
post_soup = BeautifulSoup(post, "lxml") 
#articles = soup.find_all('tr')

In [66]:
print (post_soup.prettify())[:5000]

<!DOCTYPE HTML>
<html>
 <head>
  <meta charset="utf-8"/>
  <title>
   PX是否有毒？宁波镇海事件谁在推波助澜(转载)_天涯杂谈_天涯论坛_天涯社区
  </title>
  <meta content="一网友如下说：根据百度百科收录，PX是对二甲苯（Paraxylene）的简称，和苯、甲苯一样属于芳烃物质，外观为无色透明液体，稳定性较好，不溶于水，属低毒类化工品（其后说：“PX本身毒性低于酒精，但高于食盐。”），是作为合成聚酯纤维、树脂、涂料..." itemprop="description" name="description"/>
  <meta content="PX是否有毒？宁波镇海事件谁在推波助澜(转载)_天涯杂谈_天涯论坛_天涯社区" itemprop="name"/>
  <meta content="" itemprop="image"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="zzjzzpgg12" name="author"/>
  <meta content="no-transform" http-equiv="Cache-Control"/>
  <meta content="no-siteapp" http-equiv="Cache-Control"/>
  <meta content="pc" name="applicable-device"/>
  <meta content="format=xhtml; url=http://bbs.tianya.cn/m/post-free-2848995-1.shtml" name="mobile-agent"/>
  <meta content="format=html5; url=http://bbs.tianya.cn/m/post-free-2848995-1.shtml" name="mobile-agent"/>
  <meta content="format=wml; url=http://bbs.tianya.cn/m/post-free-2848995-1.shtml" name="mobile-a

In [67]:
pa = post_soup.find_all('div', {'class', 'atl-item'})
len(pa)

95

In [68]:
print pa[0]

<div _host="zzjzzpgg12" _hostid="53134970" class="atl-item host-item">
<div class="atl-content">
<div class="atl-con-hd clearfix">
<div class="atl-con-hd-l"></div>
<div class="atl-con-hd-r"></div>
</div>
<div class="atl-con-bd clearfix">
<div class="bbs-content clearfix">
							　　一网友如下说：<br/>　　根据百度百科收录，PX是对二甲苯（Paraxylene）的简称，和苯、甲苯一样属于芳烃物质，外观为无色透明液体，稳定性较好，不溶于水，属低毒类化工品（其后说：“PX本身毒性低于酒精，但高于食盐。”），是作为合成聚酯纤维、树脂、涂料、染料和农药等的原料。<br/>　　在厦门PX事件中，被公众广为传播的一条短信这样描述PX的危害：建造PX项目意味着放了一颗原子弹，厦门人民以后将在白血病中度过。<br/>　　苯、芳烃、有机物等标签，让人们将PX与“高致癌物”联系在一起，但各种数据表明，PX其实是一种低毒性的化学品。笔者在此援引《科学时报》记者的调查结果：中科院化学所的实验室有机化学方向的一位博士生表示，“PX的毒性并不大，在有机化学中，PX是实验室中常用的有机溶剂；甚至有时候还用来擦洗物体上残留的顽固污渍。”。该博士生介绍的国际化学品安全说明书（MSDS）也显示，PX属低毒类化学物质。如果用“半数致死剂量（LD50）”的方法进行准确衡量，PX的大鼠口服LD50为5克/千克，常见的酒精和食盐的LD50值分别为7.06克/千克和3克/千克。<br/>　　也就是说，PX本身毒性低于酒精，但高于食盐。<br/>　　同样，美国得克萨斯州拉马尔大学化工系教授郭约翰表示，在美国，政府工业卫生学家会议（ACGIH）将PX归类为A4级，目前为止缺乏对人体、动物致癌性的证据。<br/>　　由此看来，PX并非一些媒体所称PX是剧毒化工品，专业知识的匮乏令一些媒体人云亦云，没有进行深一步的调查；用事实说话仍是媒体的立身之本。<br/>　　三、事件深层分析 据一位多年从事PX行业的业内人士表示，原本PX项目是很普通的

In [69]:
print pa[1]

<div _host="zzjzzpgg12" _hostid="53134970" class="atl-item" id="1" js_restime="2012-10-28 21:14:25" js_username="zzjzzpgg12" replyid="92733518">
<div class="atl-head" id="8cf559da49440878e76b86d047510091">
<div class="atl-head-reply"></div>
<div class="atl-info">
<span><strong class="host">楼主</strong>：<a class="js-vip-check" href="http://www.tianya.cn/53134970" target="_blank" uid="53134970" uname="zzjzzpgg12">zzjzzpgg12</a> </span>
<span>时间：2012-10-28 21:14:25</span>
</div>
</div>
<div class="atl-content">
<div class="atl-con-hd clearfix">
<div class="atl-con-hd-l"></div>
<div class="atl-con-hd-r"></div>
</div>
<div class="atl-con-bd clearfix">
<div class="bbs-content">
							　　百度：<br/>　　【熔点（℃）】13.2<br/>　　【沸点（℃）】138.5<br/>　　即正常情况下是液体？就是和水一样？<br/>　　在包括美国澳洲在内的很多国家,PX不算危险化学品，不过欧盟把PX列为有害品,是因为当人体吸入过量PX时,对眼及上呼吸道有刺激作用[1]。无论是危险标记、健康危害性、毒理学资料，还是在职业灾害防护等标准下，PX都不属高危高毒产品。长期反复直接接触PX会使皮肤脱脂，并有可能对中枢神经系统或听力有影响，也可能对人类生殖或发育造成毒性。国际评估化学品致癌的权威机构（IARC）对包括PX在内的整个二甲苯类的评估结果表明，PX致癌性证据尚不足，属于第三类致癌物质，即缺乏对人体致癌性证据的

In [70]:
print pa[89]

<div _host="宗族群" _hostid="73651072" class="atl-item" id="93" js_restime="2012-10-29 09:22:47" js_username="宗族群" replyid="92733622">
<div class="atl-head" id="671625cde97fe3be8715c46432b4bc18">
<div class="atl-head-reply"></div>
<div class="atl-info">
<span>作者：<a class="js-vip-check" href="http://www.tianya.cn/73651072" target="_blank" uid="73651072" uname="宗族群">宗族群</a> </span>
<span>时间：2012-10-29 09:22:47</span>
</div>
</div>
<div class="atl-content">
<div class="atl-con-hd clearfix">
<div class="atl-con-hd-l"></div>
<div class="atl-con-hd-r"></div>
</div>
<div class="atl-con-bd clearfix">
<div class="bbs-content">
							　　中国现行的体制本身就是最大的毒素与污染！
							
							
						</div>
<div class="atl-reply">
<a author="宗族群" authorid="73651072" class="reportme a-link" href="javascript:void(0);" replyid="92733622" replytime="2012-10-29 09:22:47">举报</a> |
														
							<span>93楼</span> |
							<a class="a-link-2 ir-shang" floor="93" href="javascript:void(0);" title="打赏层主">
								打赏
							

作者：柠檬在追逐 时间：2012-10-28 21:33:55

　　@lice5 2012-10-28 20:37:17
  
　　作为宁波人 还是说一句：革命尚未成功 同志仍需努力 
  
　　-----------------------------
  
　　对 现在说成功还太乐观，就怕说一套做一套

作者：lice5 时间：2012-10-28 20:37:17
        
　　作为宁波人 还是说一句：革命尚未成功 同志仍需努力

4	/post-free-4242156-1.shtml	2014-04-09 15:55:35	61943225	野渡自渡人	@Y雷政府34楼2014-04-0422:30:34　　野渡君雄文！支持是必须的。　　-----------------------------　　@清坪过客16楼2014-04-0804:09:48　　绝对的权力导致绝对的腐败！　　-----------------------------　　@T大漠鱼T35楼2014-04-0810:17:27　　@周丕东@普欣@拾月霜寒2012@小摸包@姚文嚼字@四號@凌宸@乔志峰@野渡自渡人@曾兵2010@缠绕夜色@曾颖@风青扬请关注

In [71]:
print pa[0].find('div', {'class', 'bbs-content'}).text.strip()

一网友如下说：　　根据百度百科收录，PX是对二甲苯（Paraxylene）的简称，和苯、甲苯一样属于芳烃物质，外观为无色透明液体，稳定性较好，不溶于水，属低毒类化工品（其后说：“PX本身毒性低于酒精，但高于食盐。”），是作为合成聚酯纤维、树脂、涂料、染料和农药等的原料。　　在厦门PX事件中，被公众广为传播的一条短信这样描述PX的危害：建造PX项目意味着放了一颗原子弹，厦门人民以后将在白血病中度过。　　苯、芳烃、有机物等标签，让人们将PX与“高致癌物”联系在一起，但各种数据表明，PX其实是一种低毒性的化学品。笔者在此援引《科学时报》记者的调查结果：中科院化学所的实验室有机化学方向的一位博士生表示，“PX的毒性并不大，在有机化学中，PX是实验室中常用的有机溶剂；甚至有时候还用来擦洗物体上残留的顽固污渍。”。该博士生介绍的国际化学品安全说明书（MSDS）也显示，PX属低毒类化学物质。如果用“半数致死剂量（LD50）”的方法进行准确衡量，PX的大鼠口服LD50为5克/千克，常见的酒精和食盐的LD50值分别为7.06克/千克和3克/千克。　　也就是说，PX本身毒性低于酒精，但高于食盐。　　同样，美国得克萨斯州拉马尔大学化工系教授郭约翰表示，在美国，政府工业卫生学家会议（ACGIH）将PX归类为A4级，目前为止缺乏对人体、动物致癌性的证据。　　由此看来，PX并非一些媒体所称PX是剧毒化工品，专业知识的匮乏令一些媒体人云亦云，没有进行深一步的调查；用事实说话仍是媒体的立身之本。　　三、事件深层分析 据一位多年从事PX行业的业内人士表示，原本PX项目是很普通的化工项目，但这次大连PX化工厂事件影响如此之大，很可能在这个事件背后有一股无形的力量将大连PX化工厂推上了风口浪尖，这种无形的力量很可能来自国际竞争对手。了解PX上下游产业的人士都了解，PX是一种非常重要的有机化工原料，主要用于精对苯二甲酸（PTA）生产，PTA是生产聚酯的主要原料；其中，聚酯中的聚对苯二甲酸乙二酯（PET）就是用来制造饮料瓶的材料，PET进一步经纺丝等后续处理则可制成涤纶（衣服材质）。由此可见，PX的应用非常广泛，甚至从一定层面上讲，PX产业一定程度上对我国国计民生有着举足轻重的影响。但就是PX这样一种重要的化工原料，我国目前却依赖进口，受国外产业的控制。从20世纪90年代以来，中国聚酯行业迅速发展，带动PX

In [72]:
print pa[87].find('div', {'class', 'bbs-content'}).text.strip()

回复第71楼(作者:@灰衣公子 于 2012-10-29 00:52) 　　楼主：六朝烟水满金陵 时间：2012-10-28 19:27:00 点击：4913 回复：112…… ==========　　总有人觉得自己与众不同，是在高高在上的看着别人。我不知道这位网友是在装，还是真那么高位。也许在你这样看别人是垃圾的时候，别人也是这样在看你的


In [73]:
pa[1].a

<a class="js-vip-check" href="http://www.tianya.cn/53134970" target="_blank" uid="53134970" uname="zzjzzpgg12">zzjzzpgg12</a>

In [74]:
print pa[0].find('a', class_ = 'reportme a-link')

<a author="zzjzzpgg12" authorid="53134970" class="reportme a-link" href="javascript:void(0);" replyid="0" replytime="2012-10-28 21:08:00"> 举报</a>


In [75]:
print pa[0].find('a', class_ = 'reportme a-link')['replytime']

2012-10-28 21:08:00


In [76]:
print pa[0].find('a', class_ = 'reportme a-link')['author']

zzjzzpgg12


In [77]:
for i in pa[:10]:
    p_info = i.find('a', class_ = 'reportme a-link')
    p_time = p_info['replytime']
    p_author_id = p_info['authorid']
    p_author_name = p_info['author']
    p_content = i.find('div', {'class', 'bbs-content'}).text.strip()
    p_content = p_content.replace('\t', '')
    print p_time, '--->', p_author_id, '--->', p_author_name,'--->', p_content, '\n'

2012-10-28 21:08:00 ---> 53134970 ---> zzjzzpgg12 ---> 一网友如下说：　　根据百度百科收录，PX是对二甲苯（Paraxylene）的简称，和苯、甲苯一样属于芳烃物质，外观为无色透明液体，稳定性较好，不溶于水，属低毒类化工品（其后说：“PX本身毒性低于酒精，但高于食盐。”），是作为合成聚酯纤维、树脂、涂料、染料和农药等的原料。　　在厦门PX事件中，被公众广为传播的一条短信这样描述PX的危害：建造PX项目意味着放了一颗原子弹，厦门人民以后将在白血病中度过。　　苯、芳烃、有机物等标签，让人们将PX与“高致癌物”联系在一起，但各种数据表明，PX其实是一种低毒性的化学品。笔者在此援引《科学时报》记者的调查结果：中科院化学所的实验室有机化学方向的一位博士生表示，“PX的毒性并不大，在有机化学中，PX是实验室中常用的有机溶剂；甚至有时候还用来擦洗物体上残留的顽固污渍。”。该博士生介绍的国际化学品安全说明书（MSDS）也显示，PX属低毒类化学物质。如果用“半数致死剂量（LD50）”的方法进行准确衡量，PX的大鼠口服LD50为5克/千克，常见的酒精和食盐的LD50值分别为7.06克/千克和3克/千克。　　也就是说，PX本身毒性低于酒精，但高于食盐。　　同样，美国得克萨斯州拉马尔大学化工系教授郭约翰表示，在美国，政府工业卫生学家会议（ACGIH）将PX归类为A4级，目前为止缺乏对人体、动物致癌性的证据。　　由此看来，PX并非一些媒体所称PX是剧毒化工品，专业知识的匮乏令一些媒体人云亦云，没有进行深一步的调查；用事实说话仍是媒体的立身之本。　　三、事件深层分析 据一位多年从事PX行业的业内人士表示，原本PX项目是很普通的化工项目，但这次大连PX化工厂事件影响如此之大，很可能在这个事件背后有一股无形的力量将大连PX化工厂推上了风口浪尖，这种无形的力量很可能来自国际竞争对手。了解PX上下游产业的人士都了解，PX是一种非常重要的有机化工原料，主要用于精对苯二甲酸（PTA）生产，PTA是生产聚酯的主要原料；其中，聚酯中的聚对苯二甲酸乙二酯（PET）就是用来制造饮料瓶的材料，PET进一步经纺丝等后续处理则可制成涤纶（衣服材质）。由此可见，PX的应用非常广泛，甚至从一定层面上讲，PX产业一定程度上对我国国计民生有着举足轻重的影响。但就是PX这样一

# 如何翻页

http://bbs.tianya.cn/post-free-2848797-1.shtml
    
    http://bbs.tianya.cn/post-free-2848797-2.shtml
        
        http://bbs.tianya.cn/post-free-2848797-3.shtml

In [78]:
post_soup.find('div', {'class', 'atl-pages'})#.['onsubmit']

<div class="atl-pages"><form action="" method="get" onsubmit="return goPage(this,'free',2848995,9);">\n<span>\u4e0a\u9875</span>\n<strong>1</strong>\n<a href="/post-free-2848995-2.shtml">2</a>\n<a href="/post-free-2848995-3.shtml">3</a>\n<a href="/post-free-2848995-4.shtml">4</a>\n\u2026\n<a href="/post-free-2848995-9.shtml">9</a>\n<a class="js-keyboard-next" href="/post-free-2848995-2.shtml">\u4e0b\u9875</a>\n\xa0\u5230<input class="pagetxt" name="page" type="text"/>\u9875\xa0<input class="pagebtn" maxlength="6" name="gopage" type="submit" value="\u786e\u5b9a"/></form></div>

In [79]:
post_pages = post_soup.find('div', {'class', 'atl-pages'})
post_pages = post_pages.form['onsubmit'].split(',')[-1].split(')')[0]
post_pages
#post_soup.select('.atl-pages')[0].select('form')[0].select('onsubmit')


'9'

In [80]:
url = 'http://bbs.tianya.cn' + df.link[2]
url_base = ''.join(url.split('-')[:-1]) + '-%d.shtml'
url_base

'http://bbs.tianya.cn/postfree2848995-%d.shtml'

In [102]:
def parsePage(pa):
    records = []
    for i in pa:
        p_info = i.find('a', class_ = 'reportme a-link')
        p_time = p_info['replytime']
        p_author_id = p_info['authorid']
        p_author_name = p_info['author']
        p_content = i.find('div', {'class', 'bbs-content'}).text.strip()
        p_content = p_content.replace('\t', '').replace('\n', '')#.replace(' ', '')
        record = p_time + '\t' + p_author_id+ '\t' + p_author_name + '\t'+ p_content
        records.append(record)
    return records

import sys
def flushPrint(s):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % s)
    sys.stdout.flush()

In [103]:
url_1 = 'http://bbs.tianya.cn' + df.link[10]
content = urllib2.urlopen(url_1).read() #获取网页的html文本
post_soup = BeautifulSoup(content, "lxml") 
pa = post_soup.find_all('div', {'class', 'atl-item'})
b = post_soup.find('div', class_= 'atl-pages')
b

<div class="atl-pages host-pages"></div>

In [104]:
url_1 = 'http://bbs.tianya.cn' + df.link[0]
content = urllib2.urlopen(url_1).read() #获取网页的html文本
post_soup = BeautifulSoup(content, "lxml") 
pa = post_soup.find_all('div', {'class', 'atl-item'})
a = post_soup.find('div', {'class', 'atl-pages'})
a

<div class="atl-pages"><form action="" method="get" onsubmit="return goPage(this,'free',2839539,11);">\n<span>\u4e0a\u9875</span>\n<strong>1</strong>\n<a href="/post-free-2839539-2.shtml">2</a>\n<a href="/post-free-2839539-3.shtml">3</a>\n<a href="/post-free-2839539-4.shtml">4</a>\n\u2026\n<a href="/post-free-2839539-11.shtml">11</a>\n<a class="js-keyboard-next" href="/post-free-2839539-2.shtml">\u4e0b\u9875</a>\n\xa0\u5230<input class="pagetxt" name="page" type="text"/>\u9875\xa0<input class="pagebtn" maxlength="6" name="gopage" type="submit" value="\u786e\u5b9a"/></form></div>

In [105]:
a.form

<form action="" method="get" onsubmit="return goPage(this,'free',2839539,11);">\n<span>\u4e0a\u9875</span>\n<strong>1</strong>\n<a href="/post-free-2839539-2.shtml">2</a>\n<a href="/post-free-2839539-3.shtml">3</a>\n<a href="/post-free-2839539-4.shtml">4</a>\n\u2026\n<a href="/post-free-2839539-11.shtml">11</a>\n<a class="js-keyboard-next" href="/post-free-2839539-2.shtml">\u4e0b\u9875</a>\n\xa0\u5230<input class="pagetxt" name="page" type="text"/>\u9875\xa0<input class="pagebtn" maxlength="6" name="gopage" type="submit" value="\u786e\u5b9a"/></form>

In [85]:
if b.form:
    print 'true'
else:
    print 'false'

false


In [106]:
import random
import time

def crawler(url, file_name):
    try:
        # open the browser
        url_1 = 'http://bbs.tianya.cn' + url
        content = urllib2.urlopen(url_1).read() #获取网页的html文本
        post_soup = BeautifulSoup(content, "lxml") 
        # how many pages in a post
        post_form = post_soup.find('div', {'class', 'atl-pages'})
        if post_form.form:
            post_pages = post_form.form['onsubmit'].split(',')[-1].split(')')[0]
            post_pages = int(post_pages)
            url_base = '-'.join(url_1.split('-')[:-1]) + '-%d.shtml'
        else:
            post_pages = 1
        # for the first page
        pa = post_soup.find_all('div', {'class', 'atl-item'})
        records = parsePage(pa)
        with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
            for record in records:    
                p.write('1'+ '\t' + url + '\t' + record.encode('utf-8')+"\n") 
        # for the 2nd+ pages
        if post_pages > 1:
            for page_num in range(2, post_pages+1):
                time.sleep(random.random())
                flushPrint(page_num)
                url2 =url_base  % page_num
                content = urllib2.urlopen(url2).read() #获取网页的html文本
                post_soup = BeautifulSoup(content, "lxml") 
                pa = post_soup.find_all('div', {'class', 'atl-item'})
                records = parsePage(pa)
                with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
                    for record in records:    
                        p.write(str(page_num) + '\t' +url + '\t' + record.encode('utf-8')+"\n") 
        else:
            pass
    except Exception, e:
        print e
        pass

# 测试

In [107]:
url = df.link[2]
file_name = '/Users/chengjun/github/cjc/data/tianya_bbs_threads_4test.txt'
crawler(url, file_name)

9

# 正式抓取！

In [108]:
for k, link in enumerate(df.link):
    flushPrint(link)
    if k % 10== 0:
        print 'This it the post of : ' + str(k)
    file_name = '/Users/chengjun/github/cjc/data/tianya_bbs_threads_network_2017.txt'
    crawler(link, file_name)

/post-free-2839539-1.shtmlThis it the post of : 0
/post-free-923115-1.shtmlThis it the post of : 10
/post-free-944280-1.shtmlThis it the post of : 20
/post-free-3324751-1.shtmlThis it the post of : 30
/post-free-3361796-1.shtmlThis it the post of : 40
/post-free-3533685-1.shtmlThis it the post of : 50
/post-free-4242705-1.shtmlThis it the post of : 60
/post-free-3291045-1.shtmlThis it the post of : 70
/post-free-1066741-1.shtmlThis it the post of : 80
/post-free-3312925-1.shtmlThis it the post of : 90
/post-free-3834860-1.shtmlThis it the post of : 100
/post-free-1079222-1.shtmlThis it the post of : 110
/post-free-4236159-1.shtmlThis it the post of : 120
/post-free-3282507-1.shtmlThis it the post of : 130
/post-free-3394373-1.shtmlThis it the post of : 140
/post-free-3352554-1.shtmlThis it the post of : 150
/post-free-3294035-1.shtmlThis it the post of : 160
/post-free-3305558-1.shtmlThis it the post of : 170
/post-free-3228423-1.shtmlThis it the post of : 180
/post-free-3422932-1.shtm

# 读取数据

In [109]:
dtt = []
with open('/Users/chengjun/github/cjc/data/tianya_bbs_threads_network_2017.txt', 'r') as f:
    for line in f:
        pnum, link, time, author_id, author, content = line.replace('\n', '').split('\t')
        dtt.append([pnum, link, time, author_id, author, content])
len(dtt)

5277

In [110]:
dt = pd.DataFrame(dtt)
dt[:5]

,0,1,2,3,4,5
0,1,/post-free-2839539-1.shtml,2012-10-24 12:41:00,74341835,无上卫士ABC,2012-10-24 10:42:47 来源: 人民网(北京) 核心提示：10月初，宁波...
1,1,/post-free-2839539-1.shtml,2012-10-24 13:26:06,74422795,16号渔政船,对，当官的住两年试试我们就信了
2,1,/post-free-2839539-1.shtml,2012-10-24 14:02:37,15068828,fossil2918,广大网民理性表达网络民意，不传播、不轻信网上不实信息，共同维护社会和谐稳定大局。
3,1,/post-free-2839539-1.shtml,2012-10-25 14:25:31,75446299,一言难尽XCSTZ,这样啊
4,1,/post-free-2839539-1.shtml,2012-10-25 14:49:59,18135446,owekoker07298,关注


In [111]:
dt=dt.rename(columns = {0:'page_num', 1:'link', 2:'time', 3:'author',4:'author_name', 5:'reply'})
dt[:5]

,page_num,link,time,author,author_name,reply
0,1,/post-free-2839539-1.shtml,2012-10-24 12:41:00,74341835,无上卫士ABC,2012-10-24 10:42:47 来源: 人民网(北京) 核心提示：10月初，宁波...
1,1,/post-free-2839539-1.shtml,2012-10-24 13:26:06,74422795,16号渔政船,对，当官的住两年试试我们就信了
2,1,/post-free-2839539-1.shtml,2012-10-24 14:02:37,15068828,fossil2918,广大网民理性表达网络民意，不传播、不轻信网上不实信息，共同维护社会和谐稳定大局。
3,1,/post-free-2839539-1.shtml,2012-10-25 14:25:31,75446299,一言难尽XCSTZ,这样啊
4,1,/post-free-2839539-1.shtml,2012-10-25 14:49:59,18135446,owekoker07298,关注


In [112]:
dt.reply[:100]

0     2012-10-24 10:42:47　来源: 人民网(北京)　　　核心提示：10月初，宁波...
1                                       对，当官的住两年试试我们就信了
2              广大网民理性表达网络民意，不传播、不轻信网上不实信息，共同维护社会和谐稳定大局。
3                                                   这样啊
4                                                    关注
5     水浒一块钱一本买一送三　　水浒一块钱一本买一送三　　水浒一块钱一本买一送三　　水浒一块钱一本...
6                                        我们不要化工，我们要净土！！
7                             我在八卦发的帖子被封口，不能留言，连看都不能看！！
8     @想吃甜枣 22楼 　　PX，请你滚出宁波！！　　---------------------...
9     @fossil2918 3楼 　　广大网民理性表达网络民意，不传播、不轻信网上不实信息，共同...
10            政府引进这种类似项目，我们知情么？有选择么？不过日本人侵华，也不会经过中国人同意…
11    镇海中学领导对老师已经下达了禁言令：好好教书上课，其他的不该管的不要管，不该说的不要说，学校...
12                               已经禁止镇海私家车开进市区了，违者扣12分！
13    回复第54楼(作者:@小桃子mama 于 2012-10-26 10:07) 　　作者：一笑...
14                      宁波人过几年要绝种了，水和空气都是有毒的，生出来的孩子崎形的。
15                                       江苏上支持你们一定要抗争到底
16                                            光上网叫唤有鸡吧用
17                                            湖北

## 总帖数是多少？

http://search.tianya.cn/bbs?q=PX  共有18459 条内容

In [93]:
18459/50

369

实际上到第10页就没有了 http://bbs.tianya.cn/list.jsp?item=free&order=1&nextid=9&k=PX, 原来那只是天涯论坛，还有其它各种版块，如天涯聚焦：  http://focus.tianya.cn/  等等。

- 娱乐八卦 512
- 股市论坛 187
- 情感天地 242
- 天涯杂谈 1768 

在天涯杂谈搜索雾霾，有41页 http://bbs.tianya.cn/list.jsp?item=free&order=20&nextid=40&k=%E9%9B%BE%E9%9C%BE

# 天涯SDK

http://open.tianya.cn/wiki/index.php?title=SDK%E4%B8%8B%E8%BD%BD